In [1]:
%matplotlib inline

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

pd.set_option('display.max_rows', 12)
pd.set_option('display.precision', 2)
pd.set_option('display.float_format', '{:20,.2f}'.format)

In [2]:
auto = pd.read_csv('Auto.csv', na_values=['?'])
auto.dropna(axis=0, inplace=True)

auto.cylinders = auto.cylinders.astype('category')
auto.name = auto.name.astype('category')

auto.reset_index(inplace=True)

auto['horsepower_2'] = np.power(auto.horsepower, 2)
auto['horsepower_3'] = np.power(auto.horsepower, 3)
auto['horsepower_4'] = np.power(auto.horsepower, 4)
auto['horsepower_5'] = np.power(auto.horsepower, 5)

from sklearn.preprocessing import PolynomialFeatures
pol = PolynomialFeatures(degree=5, include_bias=False, interaction_only=False)
polf = pol.fit_transform(auto.loc[:, 'horsepower'].values.reshape(-1, 1))

# Validation Set approach

In [3]:
from sklearn.model_selection import train_test_split

X, y = auto.loc[:, ['horsepower', 'horsepower_2', 'horsepower_3']], auto.mpg
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=1)

In [4]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

lm1 = LinearRegression(fit_intercept=True)
lm2 = LinearRegression(fit_intercept=True)
lm3 = LinearRegression(fit_intercept=True)

lm1_fit = lm1.fit(X_train.loc[:, 'horsepower'].values.reshape(-1, 1), y_train)
lm2_fit = lm2.fit(X_train.loc[:, ['horsepower', 'horsepower_2']], y_train)
lm3_fit = lm3.fit(X_train.loc[:, ['horsepower', 'horsepower_2', 'horsepower_3']], y_train)

lm1_predict = lm1.predict(X_test.loc[:, 'horsepower'].values.reshape(-1, 1))
lm2_predict = lm2.predict(X_test.loc[:, ['horsepower', 'horsepower_2']])
lm3_predict = lm3.predict(X_test.loc[:, ['horsepower', 'horsepower_2', 'horsepower_3']])

print('lm1 MSE: ', mean_squared_error(y_test, lm1_predict))
print('lm2 MSE: ', mean_squared_error(y_test, lm2_predict))
print('lm3 MSE: ', mean_squared_error(y_test, lm3_predict))

lm1 MSE:  24.80212062059356
lm2 MSE:  18.848292603275663
lm3 MSE:  18.805111358605085


# LOOCV

In [5]:
from sklearn.model_selection import LeaveOneOut

X, y = auto.loc[:, ['horsepower', 'horsepower_2', 'horsepower_3', 'horsepower_4', 'horsepower_5']], auto.mpg

loocv = LeaveOneOut()
loocv.get_n_splits(X)

392

In [8]:
loocv_mse = []
lm = LinearRegression(fit_intercept=True)

for train_index, test_index in loocv.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    lm1_fit = lm.fit(X_train.loc[:, 'horsepower'].values.reshape(-1, 1), y_train)
    lm1_predict = lm.predict(X_test.loc[:, 'horsepower'].values.reshape(-1, 1))
    
    loocv_mse.append(mean_squared_error(y_test, lm1_predict))
    
np.array(loocv_mse).mean()

24.231513517929226

In [9]:
from sklearn.model_selection import cross_val_score

lm = LinearRegression(fit_intercept=True)
cval = cross_val_score(lm, 
                       auto.loc[:, 'horsepower'].values.reshape(-1, 1),
                       auto.mpg,
                       cv = len(auto),
                       n_jobs=-1,
                       scoring='neg_mean_squared_error')
-cval.mean()

24.231513517929226

In [10]:
loocv_poly = {}

for i in range(1, 6):
    loocv_mse = []
    for train_index, test_index in loocv.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
        if i==1:
            X_TRAIN = X_train.iloc[:,0:1].values.reshape(-1, 1)
            X_TEST = X_test.iloc[:,0:1].values.reshape(-1, 1)
        else:
            X_TRAIN = X_train.iloc[:,0:i]
            X_TEST = X_test.iloc[:,0:i]
            
        loocv_mse.append(
            mean_squared_error(
                y_test,
                LinearRegression(fit_intercept=True).fit(X_TRAIN, y_train).predict(X_TEST)
            )
        )
        
        loocv_poly['lm' + str(i) + '_mse'] = np.array(loocv_mse).mean()

# K-FOLD

In [11]:
from sklearn.model_selection import KFold

X, y = auto.loc[:, ['horsepower', 'horsepower_2', 'horsepower_3', 'horsepower_4', 'horsepower_5']], auto.mpg
kf = KFold(n_splits=10, shuffle=True, random_state=1)

In [12]:
kf_poly = {}

for i in range(1,6):
    kf_mse = []
    
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        if i==1:
            X_TRAIN = X_train.iloc[:,0:1].values.reshape(-1, 1)
            X_TEST = X_test.iloc[:,0:1].values.reshape(-1, 1)
        else:
            X_TRAIN = X_train.iloc[:,0:i]
            X_TEST = X_test.iloc[:,0:i]
        
        kf_mse.append(
            mean_squared_error(
                y_test,
                LinearRegression(fit_intercept=True).fit(X_TRAIN, y_train).predict(X_TEST)
            )
        )
        
        kf_poly['lm' + str(i) + '_mse'] = np.array(kf_mse).mean()

# Bootstrap

In [13]:
portfolio = pd.read_csv('Portfolio.csv', index_col=0)

def alpha_fn(data, start_index, end_index):
    X = data['X'].iloc[start_index:end_index]
    Y = data['Y'].iloc[start_index:end_index]
    return ((np.var(Y) - np.cov(X, Y)[0,1]) / np.var(X) + np.var(Y) - 2 * np.cov(X,Y)[0][1])

alpha_fn(portfolio, 0, 100)

0.6409915594550861

In [14]:
from sklearn.utils import resample

portfolio_bs = resample(portfolio, replace=True, n_samples=100, random_state=1)
alpha_fn(portfolio_bs, 0, 100)

0.62690992488482

In [16]:
bs_alpha = []

for i in range(0, 1000):
    bs_alpha.append(alpha_fn(resample(portfolio, replace=True, n_samples=100), 0, 100))
    
bs_alpha = np.array(bs_alpha)

print('Bootstrapped alpha: ', bs_alpha.mean())
print('SE: ', bs_alpha.std())

Bootstrapped alpha:  0.6428256429814548
SE:  0.30276186859309523


In [17]:
def boot_fn(data, start_index, end_index, n_iter=1000):
    m = LinearRegression(fit_intercept=True).fit(
        data['horsepower'].iloc[start_index:end_index].values.reshape(-1, 1),
        data['mpg'].iloc[start_index:end_index]
    )
    
    return m.intercept_, m.coef_

boot_fn(auto, 0, 392)

(39.93586102117047, array([-0.15784473]))

In [18]:
boot_fn(resample(auto, replace=True, n_samples=392), 0, 392)

(40.38247812500858, array([-0.16216735]))

In [19]:
bs_boot = {'t1': [], 't2': []}

for i in range(0, 1000):
    bs_boot['t1'].append(boot_fn(resample(auto, replace=True, n_samples=392), 0, 392)[0])
    bs_boot['t2'].append(boot_fn(resample(auto, replace=True, n_samples=392), 0, 392)[1][0])
    
t1_es = np.array(bs_boot['t1']).mean()
t1_se = np.array(bs_boot['t1']).std()
t2_es = np.array(bs_boot['t2']).mean()
t2_se = np.array(bs_boot['t2']).std()

print('t1 bs estimate & se: ', t1_es, t1_se)
print('t2 bs estimate & se: ', t2_es, t2_se)

t1 bs estimate & se:  39.96718036183598 0.8504957393115411
t2 bs estimate & se:  -0.15823825986602216 0.007205367474335722


In [20]:
import statsmodels.formula.api as sm

ols = sm.ols('mpg ~ horsepower', data=auto).fit()
ols.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,39.9359,0.717,55.660,0.000,38.525,41.347
horsepower,-0.1578,0.006,-24.489,0.000,-0.171,-0.145


In [21]:
def boot_fn2(data, start_index, end_index, n_iter=1000):
    m = LinearRegression(fit_intercept=True).fit(
        data[['horsepower', 'horsepower_2']].iloc[start_index:end_index],
        data['mpg'].iloc[start_index:end_index]
    )

    return m.intercept_, m.coef_

In [22]:
bs_boot2 = {'t1': [], 't2': [], 't3': []}

for i in range(0, 1000):
    bs_boot2['t1'].append(boot_fn2(resample(auto, replace=True, n_samples=392), 0, 392)[0])
    bs_boot2['t2'].append(boot_fn2(resample(auto, replace=True, n_samples=392), 0, 392)[1][0])
    bs_boot2['t3'].append(boot_fn2(resample(auto, replace=True, n_samples=392), 0, 392)[1][1])
    
t1_es = np.array(bs_boot2['t1']).mean()
t1_se = np.array(bs_boot2['t1']).std()
t2_es = np.array(bs_boot2['t2']).mean()
t2_se = np.array(bs_boot2['t2']).std()
t3_es = np.array(bs_boot2['t3']).mean()
t3_se = np.array(bs_boot2['t3']).std()

print('t1 bs estimate & se: ', t1_es, t1_se)
print('t2 bs estimate & se: ', t2_es, t2_se)
print('t3 bs estimate & se: ', t3_es, t3_se)

t1 bs estimate & se:  56.97936636708295 2.102068074259394
t2 bs estimate & se:  -0.46773479857707695 0.033940347548895536
t3 bs estimate & se:  0.001240305355354604 0.00012150477155297943


In [23]:
ols2 = sm.ols('mpg ~ horsepower + horsepower_2', data=auto).fit()
ols2.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,56.9001,1.800,31.604,0.000,53.360,60.440
horsepower,-0.4662,0.031,-14.978,0.000,-0.527,-0.405
horsepower_2,0.0012,0.000,10.080,0.000,0.001,0.001
